In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from tensorflow.keras import datasets, layers, models, backend, optimizers, losses
class simple_mpl(object):
    def __init__(self, path, file_head):
        print("Loading", path, "+", file_head)
        file_list = os.listdir(path) 
        num = len(file_head + "_")
        self.models = []
        for file in file_list:
            if file[:num] == file_head + "_":
                self.models.append(tf.keras.models.load_model(os.path.join(path, file)))
        cfg = pd.read_pickle(os.path.join(path, 'config.pkl'))
        self.mean = cfg['mean']
        self.std =cfg['std']
        self.output_num = cfg["output_num"].max()
    def predict(self, data):
        pred = pd.DataFrame(np.zeros([data.index.size, self.output_num]), index=data.index)
        for mdl in self.models:                                 
            pred += mdl.predict((data.astype("float32") - self.mean)/ self.std, batch_size=4096)
        pred /= len(self.models)
        return pred

In [ ]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
train_target = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test = pd.read_csv('../input/lish-moa/test_features.csv')
targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
#one-hot encoding
train_size = train.shape[0]
traintest = pd.concat([train, test])
#traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_type'], prefix='cp_type')], axis=1)
traintest.drop(['cp_type'], axis=1, inplace=True)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_time'], prefix='cp_time')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_dose'], prefix='cp_dose')], axis=1)
traintest = traintest.drop(['cp_time', 'cp_dose'], axis=1)
train = traintest[:train_size]
test  = traintest[train_size:]
del traintest
x_train = train.drop('sig_id', axis=1)
y_train = train_target.drop('sig_id', axis=1).astype(np.float32)
x_test = test.drop('sig_id', axis=1)

In [ ]:
mdl = simple_mpl(path="../input/mplmodel11/mpl", file_head="")
remove_cols=['g-39', 'g-85', 'g-75', 'g-734', 'g-484', 'g-217', 'g-545', 'g-10']
x_test = x_test[np.setdiff1d(x_test.columns.values, remove_cols)]
res = mdl.predict(x_test)
submit = pd.DataFrame(res.values, index = test['sig_id'])
submit.columns = y_train.columns
submit.to_csv("submission.csv")
submit